In [ ]:
!pip install -r requirements.txt

In [1]:
from accelerate.utils import write_basic_config

write_basic_config()

PosixPath('/home/user/.cache/huggingface/accelerate/default_config.yaml')

In [5]:
!accelerate launch train.py --mixed_precision "fp16" --gradient_accumulation_steps 1 --learning_rate "1e-02" --gradient_checkpointing --max_grad_norm 1 --checkpointing_steps 100000 --max_train_steps=15000 --use_ema --lr_scheduler "constant" --lr_warmup_steps 0 --center_crop --random_flip --pretrained_model_name_or_path "runwayml/stable-diffusion-v1-5" --dataset_name "nroggendorff/zelda" --resolution 512 --train_batch_size 1 --output_dir "sd-zelda-model"

04/27/2024 20:13:47 - INFO - __main__ - Distributed environment: NO
Num processes: 1
Process index: 0
Local process index: 0
Device: cuda

Mixed precision type: fp16

{'sample_max_value', 'variance_type', 'thresholding', 'rescale_betas_zero_snr', 'timestep_spacing', 'clip_sample_range', 'dynamic_thresholding_ratio', 'prediction_type'} was not found in config. Values will be initialized to default values.
{'scaling_factor', 'force_upcast', 'latents_std', 'latents_mean'} was not found in config. Values will be initialized to default values.
{'addition_embed_type_num_heads', 'time_embedding_type', 'upcast_attention', 'dropout', 'mid_block_type', 'class_embed_type', 'resnet_time_scale_shift', 'transformer_layers_per_block', 'resnet_out_scale_factor', 'addition_time_embed_dim', 'use_linear_projection', 'only_cross_attention', 'reverse_transformer_layers_per_block', 'time_embedding_dim', 'class_embeddings_concat', 'time_cond_proj_dim', 'mid_block_only_cross_attention', 'timestep_post_act', '

In [ ]:
from huggingface_hub import upload_folder, notebook_login, create_repo

In [ ]:
notebook_login()

In [ ]:
repo_id = create_repo(
    repo_id="nroggendorff/zelda-diffusion", exist_ok=True
).repo_id

In [ ]:
upload_folder(
    repo_id=repo_id,
    folder_path="sd-zelda-model",
    commit_message=f"Steps 10k",
    ignore_patterns=["step_*", "epoch_*"],
)

In [ ]:
from diffusers import StableDiffusionPipeline
import torch

pipeline = StableDiffusionPipeline.from_pretrained("sd-zelda-model", torch_dtype=torch.float16, use_safetensors=True).to("cuda")

image = pipeline(prompt="a woman holding a sword").images[0]
image.save("zelda.png")